In [18]:
# I wanted to ask this as a question on Stackoverflow

backend_data = {
    "admins": ["Leo", "Martin", "Thomas", "Katrin"],
    "members": [
        "Leo",
        "Martin",
        "Thomas",
        "Katrin",
        "Subhayan",
        "Clemens",
        "Thoralf"
    ],
    "juniors": ["Orianne", "Antonia", "Sarah"]
}

class Backend:
    def __init__(self, data):
        self.backend_data = data
        
    def get_all_admins(self, extras=None):
        if extras is None:
            return self.backend_data.get("admins")
        data = self.backend_data.get("admins")
        data.extend(extras)
        return data
    
    def get_all_members(self):
        return self.backend_data.get("members")
    
    def get_all_juniors(self):
        return self.backend_data.get("juniors")
    
    
class BackendAdaptor:
    # Does some conversion and validation
    # For the methods get_all_juniors and get_all_admins
    # The method just delegates to backend and returns a generator of the results
    # While for the method get_all_members there is some validation
    def __init__(self, backend):
        self.backend = backend
        
#     def get_all_admins(self, extras=None):
#         return (admin for admin in self.backend.get_all_admins(extras))
    
    def get_all_members(self):
        return (member for member in self.backend.get_all_members() if member not in self.backend.get_all_admins())
    
#     def get_all_juniors(self):
#         return (junior for junior in self.backend.get_all_juniors())
    
    def __getattr__(self, name):
        if not hasattr(self.backend, name):
            raise AttributeError(f"'{name}' not in backend.")
        return lambda *args, **kwargs: (i for i in getattr(self.backend, name)(*args, **kwargs))
    
    
if __name__ == "__main__":
    backend = Backend(data=backend_data)
    adaptor = BackendAdaptor(backend=backend)
    print(f"All admins are : {list(adaptor.get_all_admins(extras=['Gregor', 'Alex']))}")
    print(f"All members are : {list(adaptor.get_all_members())}")
    print(f"All juniors are : {list(adaptor.get_all_juniors())}")
    adaptor.get_something_else() # This should raise an attribute error
        

All admins are : ['Leo', 'Martin', 'Thomas', 'Katrin', 'Gregor', 'Alex']
All members are : ['Subhayan', 'Clemens', 'Thoralf']
All juniors are : ['Orianne', 'Antonia', 'Sarah']


AttributeError: 'get_something_else' not in backend.

In [3]:
numbers = list(range(10, 100, 5))

In [4]:
numbers

[10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]

In [11]:
some_indices = [3, 6, 10, 13]
start_index = 0
for index in some_indices:
    print(f"Number at index : {numbers[index]}")
    print(numbers[start_index:index])
    start_index = index + 1
print(f"Finally : {numbers[start_index:]}")

Number at index : 25
[10, 15, 20]
Number at index : 40
[30, 35]
Number at index : 60
[45, 50, 55]
Number at index : 75
[65, 70]
Finally : [80, 85, 90, 95]


In [3]:
# Recipe to break a dataframe into chunks
# I must have used this in my expense viewer project

import itertools

def break_up_iterable_into_chunks(numbers, chunk_indices):
    start_index = 0
    for index in chunk_indices:
        yield numbers[start_index:index]
        start_index = index + 1
    yield numbers[start_index:]
    

for chunk in itertools.islice(break_up_iterable_into_chunks(numbers=list(range(10, 100, 5)), chunk_indices=[3, 6, 10, 13]), 1, None):
    print(chunk)

[30, 35]
[45, 50, 55]
[65, 70]
[80, 85, 90, 95]


In [16]:
# Very very trivial order class  just to check
# how a property factory works

def quantity(storage):
    def qty_getter(instance):
        print("Getter function getting called")
        return instance.__dict__[storage]
    
    def qty_setter(instance, value):
        print("Setter function getting called")
        if value < 0:
            raise ValueError("Value should be more than 0")
        else:
            instance.__dict__[storage] = value
            
    return property(qty_getter, qty_setter)

class Order:
    items = quantity('items')
    discount = quantity('discount')
    
    def __init__(self, name, items, discount):
        self.name = name
        self.items = items
        self.discount = discount
        
    def calculate_total(self):
        # Lets assume that the prince of the item is 100 
        return (self.items * (100 - self.discount))
        
    

In [17]:
c = Order("Vegetables", 20, 10)

Setter function getting called
Setter function getting called


In [18]:
c.calculate_total()

Getter function getting called
Getter function getting called


1800

In [19]:
c.items = -10

Setter function getting called


ValueError: Value should be more than 0

In [20]:
c.discount = -20

Setter function getting called


ValueError: Value should be more than 0